In [284]:
import numpy as np
import pandas as pd
import pandas_ta as ta
import datetime
from tqdm import tqdm
tqdm.pandas()

import csv

In [326]:
df  = pd.read_csv('../Datasets/Binance_BTCUSDT_d.csv', header=1, usecols=['unix','open','high','low','close', 'Volume BTC'], nrows=1680)
df.dropna(inplace=True, how='any')

Transform unix into date readable

In [327]:
df['date']=pd.to_datetime(df['unix'],unit='ms')
df.drop(columns='unix', inplace=True)


In [328]:
df.set_index('date', inplace=True)

Define my first strategy

In [329]:
strat1 = ta.Strategy(name='Strategie 1',
                     description='MACD, RSI, EMA12, EMA26, Stochastic Oscillator',
                     ta=[
                       {'kind':'macd', 'fast':12 , 'slow':26},
                       {'kind':'sma', 'length':50},
                       {'kind':'sma', 'length':200},
                       {'kind':'stoch'},
                       {'kind':'rsi', 'length':5},
                       {'kind':'rsi', 'length':8},
                       {'kind':'rsi', 'length':10},
                       {'kind':'rsi', 'length':15},
                       {'kind':'rsi', 'length':14},
                       {'kind':'rsi', 'length':24},
                       {'kind':'rsi', 'length':20},
                       {'kind':'rsi', 'length':60},
                       {'kind':'rsi', 'length':95},
                       
                     ])

Apply the strategy

In [330]:
df.ta.strategy(strat1)


Remove first value because of nan

In [331]:
df = df.iloc[200:]

In [332]:
df['diff'] = df['close'] - df['open']

In [333]:
def simplifier(val):
  if(val > 50):
    return 2
  elif(val<-50):
    return 0
  return 1
df['y'] = df['diff'].progress_apply(simplifier)

100%|██████████| 1480/1480 [00:00<00:00, 737416.24it/s]


# Machine Learning

In [334]:
from sklearn.model_selection import train_test_split

In [335]:
df = df[::-1]
df['y']=df['y'].shift(-1)

In [336]:
df.columns

Index(['open', 'high', 'low', 'close', 'Volume BTC', 'MACD_12_26_9',
       'MACDh_12_26_9', 'MACDs_12_26_9', 'SMA_50', 'SMA_200', 'STOCHk_14_3_3',
       'STOCHd_14_3_3', 'RSI_5', 'RSI_8', 'RSI_10', 'RSI_15', 'RSI_14',
       'RSI_24', 'RSI_20', 'RSI_60', 'RSI_95', 'diff', 'y'],
      dtype='object')

In [337]:
df = df[:-1]

In [449]:
#X = df.drop(columns=['diff','y'])
X=df[['MACD_12_26_9', 'MACDh_12_26_9','RSI_5', 'RSI_10', 'RSI_15']] #0.6824324324324325
y = df['y']

In [450]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [451]:
from sklearn.svm import SVC
svc = SVC()
svc.fit(X_train, y_train)
svc.score(X_test, y_test)

0.4864864864864865

In [452]:
#https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html?highlight=mlp#sklearn.neural_network.MLPClassifierß
from sklearn.neural_network import MLPClassifier
parameter_space = {
    'hidden_layer_sizes': [(100,),(3,3,3), (10,10,10), (30,100,50)], #(50,50,50), (50,100,50), 
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}
mlp=MLPClassifier(max_iter=500, random_state=1, )

In [457]:
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')

clf = GridSearchCV(mlp, parameter_space, n_jobs=-1, verbose=1, cv=7)
clf.fit(X_train, y_train)

Fitting 7 folds for each of 64 candidates, totalling 448 fits


/Users/sam/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/sam/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/sam/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/sam/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/sam/Library/Pytho

GridSearchCV(cv=7, estimator=MLPClassifier(max_iter=500, random_state=1),
             n_jobs=-1,
             param_grid={'activation': ['tanh', 'relu'],
                         'alpha': [0.0001, 0.05],
                         'hidden_layer_sizes': [(100,), (3, 3, 3), (10, 10, 10),
                                                (30, 100, 50)],
                         'learning_rate': ['constant', 'adaptive'],
                         'solver': ['sgd', 'adam']},
             verbose=1)

In [458]:
# Best paramete set
print('Best parameters found:\n', clf.best_params_)

Best parameters found:
 {'activation': 'relu', 'alpha': 0.0001, 'hidden_layer_sizes': (100,), 'learning_rate': 'adaptive', 'solver': 'sgd'}


In [459]:
clf.score(X_test, y_test)

0.6891891891891891

In [460]:
#0.6891891891891891 'MACD_12_26_9', 'MACDh_12_26_9','RSI_5', 'RSI_10', 'RSI_15'
clf.predict_proba(X_test)

array([[7.08445138e-01, 2.01080129e-01, 9.04747327e-02],
       [1.24409509e-01, 1.67159494e-01, 7.08430997e-01],
       [4.16252000e-01, 2.72153165e-01, 3.11594835e-01],
       [6.71115558e-01, 2.10623041e-01, 1.18261401e-01],
       [3.48407425e-01, 2.94432088e-01, 3.57160487e-01],
       [1.66954393e-01, 1.07496650e-01, 7.25548957e-01],
       [5.67147415e-01, 1.93017016e-01, 2.39835569e-01],
       [6.07356628e-01, 2.68291038e-01, 1.24352333e-01],
       [1.08238339e-01, 9.96894393e-02, 7.92072222e-01],
       [4.99396901e-01, 1.47902995e-19, 5.00603099e-01],
       [1.97029915e-01, 7.93338777e-02, 7.23636207e-01],
       [2.70683344e-01, 9.08469675e-02, 6.38469688e-01],
       [2.73545973e-01, 3.03097552e-01, 4.23356475e-01],
       [5.07948695e-01, 2.56273070e-01, 2.35778235e-01],
       [6.08839551e-01, 2.55639003e-01, 1.35521446e-01],
       [6.88442422e-02, 1.65678845e-05, 9.31139190e-01],
       [3.23265213e-01, 5.41200394e-04, 6.76193586e-01],
       [4.77929193e-01, 1.80986

In [461]:
import pickle

In [462]:
#Save model
filename='./model/MLP689.pkl'
with open(filename, 'wb') as file:
  pickle.dump(svc, file)
  

In [176]:
#Load model
with open(filename, 'rb') as file:
    model = pickle.load(file)
